In [15]:
!pip freeze | grep scikit-learn

scikit-learn @ file:///croot/scikit-learn_1684954695550/work


In [16]:
!python -V

Python 3.11.7


In [17]:
import pickle
import pandas as pd
import numpy as np

year = 2023
month = 3
taxi_type = 'yellow'

output_file = f'output/{taxi_type}/{year:04d}-{month:02d}.parquet'

In [18]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/home/cpl/anaconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/cpl/anaconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [19]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [20]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [21]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [22]:
# Calculate the standard deviation of the predicted durations
std_dev = np.std(y_pred)
print("Standard Deviation of Predicted Durations:", std_dev)

Standard Deviation of Predicted Durations: 6.247488852238703


In [23]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')


In [25]:
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['predicted_duration'] = y_pred

In [27]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)